# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Simplest script to create a visualization of a cell

In [ ]:
from bioexplorer import BioExplorer, Protein, Membrane, Cell, Vector2, Vector3, Quaternion
be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset()

In [ ]:
be.core_api().set_camera(
    orientation=[0.0648, -0.028, 0.0,0.99],
    position=[-11.817, 38.056, 34.117],
    target=[-10.538, 41.009, 11.533]
)

In [ ]:
def add_cell(position_random_seed, orientation_randon_seed):
    
    core.set_application_parameters(image_stream_fps=0)
    representation=be.REPRESENTATION_ATOMS

    # Resources
    resource_folder = '../../tests/test_files/'
    pdb_folder = resource_folder + 'pdb/'

    # ACE2 receptor definition
    ace2_receptor = Protein(
        sources=[pdb_folder + '6m18.pdb'],
        occurences=8,
        position=Vector3(0.0, 6.0, 0.0),
        rotation=Quaternion(0.0, 1.0, 0.0, 0.0)
    )

    # Membrane definition
    membrane_size = 800.0
    membrane_amplitude = 80.0 # For sinusoidal shape only
    membrane = Membrane(
        sources=[pdb_folder + 'membrane/popc.pdb'],
        occurences=200000)

    # Cell definition
    cell = Cell(
        name='Cell',size=membrane_size,
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
        membrane=membrane, receptor=ace2_receptor,
        random_position_strength=0.1,
        random_position_seed=position_random_seed,
        random_rotation_strength=10.0,
        random_rotation_seed=orientation_randon_seed,
        extra_parameters=[membrane_amplitude]
    )

    # Add cell to scene
    be.add_cell(cell=cell, representation=representation)
    
    be.set_models_visibility(True)
    core.set_application_parameters(image_stream_fps=20)

In [ ]:
core.set_camera(
    orientation=[0.014, -0.204, -0.004, 0.978],
    position=[-219.812, 10.528, 483.105],
    target=[-39.765, 22.514, 71.92]    
)

In [ ]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 500
params.shadows = 1.0
params.soft_shadows = 0.3
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

In [ ]:
from braynsmediamaker import MovieMaker
from IPython.display import clear_output

mm = MovieMaker(core)
output_folder = '/tmp'

for frame in range(1, 20):
    clear_output()
    print('Frame %i' % frame)

    '''Cell'''
    add_cell(frame + 1, frame + 2)
    
    '''Snapshot'''
    mm.create_snapshot(
        size=[960, 540], samples_per_pixel=16,
        path=output_folder + '/%05d.png' % frame)